work in progress

 **Introduction**
 
Here I will explore the data presented by "UCI Machine Learning" for mushroom classification. The goal of this dataset is to classify between edible (e) to poisonous (p) mushrooms. First, I will analyze the data and then I will classify it to achieve high accuracy and precision. 

**Python libraries**

The libraries for this kernel are:

•	[Numpy](http://http://www.numpy.org/)

•	[Pandas](http://pandas.pydata.org/)

•	[Seaborn](https://seaborn.pydata.org/)

•	[Matplotlib](https://matplotlib.org/)

•	[Scikit-Learn](http://scikit-learn.org/stable/index.html)

Numpy will be used for linear algebra and fixing arrays, Pandas for data processing, Seaborn for cool visualizations, matplotlib for figures and Scikit-Learn for preprocessing and machine learning algorithms.
Importing of those libraries:


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

**Loading the data and initial exploration**

In [ ]:
df = pd.read_csv('../input/mushrooms.csv')

In [ ]:
df.columns

In [ ]:
df.head(5)

The data is categorial so I convert it with LabelEncoder to transfer to ordinal.

In [ ]:
labelencoder=LabelEncoder()
for column in df.columns:
    df[column] = labelencoder.fit_transform(df[column])

In [ ]:
df.describe()

From the table above it can be seen that the column "veil-type" is 0 and not contributing to the data so I remove it.

In [ ]:
df=df.drop(["veil-type"],axis=1)

**Is the data balanced?**

In [ ]:
plt.figure()
pd.Series(df['class']).value_counts().sort_index().plot(kind = 'bar')
plt.ylabel("Count")
plt.xlabel("class")
plt.title('Number of poisonous/edible mushrooms (0=edible, 1=poisonous)')

Let's look at the correlation between the variables:

In [ ]:
plt.figure(figsize=(14,12))
sns.heatmap(df.corr(),linewidths=.1,cmap="YlGnBu", annot=True)
plt.yticks(rotation=0)

Usually the least correlating variable is the most important one for classification. In this case, "gill-color" has -0.53 so let's look at it closely:

In [ ]:
df[['class', 'gill-color']].groupby(['gill-color'], as_index=False).mean().sort_values(by='class', ascending=False)

In [ ]:
df[['class', 'population']].groupby(['population'], as_index=False).mean().sort_values(by='class', ascending=False)

In [ ]:
df[['class', 'spore-print-color']].groupby(['spore-print-color'], as_index=False).mean().sort_values(by='class', ascending=False)

**Model, predict and estimate the result:**

In [ ]:
X=df.drop(['class'], axis=1)
Y=df['class']

In [ ]:
X_train, X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.1, random_state=29)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import subprocess

clf = DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [ ]:
import graphviz
dot_data = export_graphviz(clf, out_file=None,  
                         feature_names=X.columns, 
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

In [ ]:
features_list = X.columns.values
feature_importance = clf.feature_importances_
sorted_idx = np.argsort(feature_importance)
 
plt.figure(figsize=(5,7))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), features_list[sorted_idx])
plt.xlabel('Importance')
plt.title('Feature importances')
plt.draw()
plt.show()

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print("Decision Tree Classifier report \n", classification_report(Y_test, y_pred))

In [ ]:
confusion_matrix(Y_test, y_pred)